In [1]:
import xarray as xr
import numpy as np
from statsmodels.stats.multitest import fdrcorrection


In [1]:
#Apply FDR to QSW -> background correlations in ECMWF
pzu=xr.open_dataset('u_pvals_Mtemporal.nc')
pzsst=xr.open_dataset('sst_pvals_Mtemporal.nc')
pzu50=xr.open_dataset('u50_pvals_Mtemporal.nc')
pzwg=xr.open_dataset('wg_pvals_Mtemporal.nc')

pzu_st=pzu.stack(z={'latitude','longitude'})
pnew_u=pzu_st.u.copy(deep=True)
pnew_u.values=fdrcorrection(pzu_st.u.values,alpha=0.05, method='indep', is_sorted=False)[1]

pzu50_st=pzu50.stack(z={'latitude','longitude'})
pnew_u50=pzu50_st.u.copy(deep=True)
pnew_u50.values=fdrcorrection(pzu50_st.u.values,alpha=0.05, method='indep', is_sorted=False)[1]

pzsst_st=pzsst.stack(z={'latitude','longitude'})
pzsst_drna=pzsst_st.dropna(dim='z')
pnew_sst=pzsst_st.sst.copy(deep=True)
pnew_drnasst=pzsst_drna.sst.copy(deep=True)
pzfdrst=fdrcorrection(pzsst_drna.sst.values,alpha=0.05, method='indep', is_sorted=False)[1]
pnew_drnasst.values=pzfdrst
pnew_sst.loc[pnew_sst.z.isin(pnew_drnasst.z)]=pnew_drnasst

pzwg_st=pzwg.stack(z={'latitude','longitude'})
pzwg_drna=pzwg_st.dropna(dim='z')
pnew_wg=pzwg_st.ngl_pv_rz.copy(deep=True)
pnew_drnawg=pzwg_drna.ngl_pv_rz.copy(deep=True)
pzfdrst=fdrcorrection(pzwg_drna.ngl_pv_rz.values,alpha=0.05, method='indep', is_sorted=False)[1]
pnew_drnawg.values=pzfdrst
pnew_wg.loc[pnew_wg.z.isin(pnew_drnawg.z)]=pnew_drnawg

In [2]:
#SAVE to files
pnew_u.unstack().to_dataset().to_netcdf('usk_pvals_fdr_new_Mtemporal.nc')
pnew_sst.unstack().to_dataset().to_netcdf('sstsk_pvals_fdr_new_Mtemporal.nc')

pnew_u50.unstack().to_dataset().to_netcdf('u50sk_pvals_fdr_new_Mtemporal.nc')
pnew_wg.unstack().to_dataset().to_netcdf('wgsk_pvals_fdr_new_Mtemporal.nc')


In [3]:
#Apply FDR to QSW -> background correlations in ERA5

pzu=xr.open_dataset('u_pvals_Otemp.nc')
pzsst=xr.open_dataset('sst_pvals_Otemp.nc')
pzu50=xr.open_dataset('u50_pvals_Otemp.nc')
pzwg=xr.open_dataset('wg_pvals_Otemp.nc')

pzu_st=pzu.stack(z={'latitude','longitude'})
pnew_u=pzu_st.u.copy(deep=True)
pnew_u.values=fdrcorrection(pzu_st.u.values,alpha=0.05, method='indep', is_sorted=False)[1]

pzu50_st=pzu50.stack(z={'latitude','longitude'})
pnew_u50=pzu50_st.u.copy(deep=True)
pnew_u50.values=fdrcorrection(pzu50_st.u.values,alpha=0.05, method='indep', is_sorted=False)[1]

pzsst_st=pzsst.stack(z={'latitude','longitude'})
pzsst_drna=pzsst_st.dropna(dim='z')
pnew_sst=pzsst_st.sst.copy(deep=True)
pnew_drnasst=pzsst_drna.sst.copy(deep=True)
pzfdrst=fdrcorrection(pzsst_drna.sst.values,alpha=0.05, method='indep', is_sorted=False)[1]
pnew_drnasst.values=pzfdrst
pnew_sst.loc[pnew_sst.z.isin(pnew_drnasst.z)]=pnew_drnasst

pzwg_st=pzwg.stack(z={'latitude','longitude'})
pzwg_drna=pzwg_st.dropna(dim='z')
pnew_wg=pzwg_st.ngl_pv_rz.copy(deep=True)
pnew_drnawg=pzwg_drna.ngl_pv_rz.copy(deep=True)
pzfdrst=fdrcorrection(pzwg_drna.ngl_pv_rz.values,alpha=0.05, method='indep', is_sorted=False)[1]
pnew_drnawg.values=pzfdrst
pnew_wg.loc[pnew_wg.z.isin(pnew_drnawg.z)]=pnew_drnawg

In [4]:
#SAVE to files
pnew_u.unstack().to_dataset().to_netcdf('usk_pvals_fdr_new_Otemporal.nc')
pnew_sst.unstack().to_dataset().to_netcdf('sstsk_pvals_fdr_new_Otemporal.nc')

pnew_u50.unstack().to_dataset().to_netcdf('u50sk_pvals_fdr_new_Otemporal.nc')
pnew_wg.unstack().to_dataset().to_netcdf('wgsk_pvals_fdr_new_Otemporal.nc')

In [1]:
#Apply FDR to QSW skill to background correlations
pzu=xr.open_dataset('u_pvals_Mampl.nc')
pzsst=xr.open_dataset('sst_pvals_Mampl.nc')
pzu50=xr.open_dataset('u50_pvals.nc')
pzgh=xr.open_dataset('gh_pvals_Mampl.nc')

pzu_st=pzu.stack(z={'latitude','longitude'})
pnew_u=pzu_st.u.copy(deep=True)
pnew_u.values=fdrcorrection(pzu_st.u.values,alpha=0.05, method='indep', is_sorted=False)[1]

pzu50_st=pzu50.stack(z={'latitude','longitude'})
pnew_u50=pzu50_st.u.copy(deep=True)
pnew_u50.values=fdrcorrection(pzu50_st.u.values,alpha=0.05, method='indep', is_sorted=False)[1]

pzsst_st=pzsst.stack(z={'latitude','longitude'})
pnew_sst=pzsst_st.sst.copy(deep=True)
pnew_sst.values=fdrcorrection(pzsst_st.sst.values,alpha=0.05, method='indep', is_sorted=False)[1]

pzwg_st=pzwg.stack(z={'latitude','longitude'})
pnew_wg=pzwg_st.wg.copy(deep=True)
pnew_wg.values=fdrcorrection(pzwg_st.wg.values,alpha=0.05, method='indep', is_sorted=False)[1]

In [2]:
#SAVE to files
pnew_u.unstack().to_dataset().to_netcdf('u_pvals_fdr_MAMP_new.nc')
pnew_sst.unstack().to_dataset().to_netcdf('sst_pvals_fdr_MAMP_new.nc')

pnew_u50.unstack().to_dataset().to_netcdf('u50_pvals_fdr_new.nc')
pnew_wg.unstack().to_dataset().to_netcdf('wg_pvals_fdr_MAMP_new.nc')


In [7]:
#Apply FDR to t2m/blocking correlations

pztO=xr.open_dataset('t2m_pvals_Oampl.nc')
pzbO=xr.open_dataset('block_pvals_Oampl.nc')

pztM=xr.open_dataset('t2m_pvals_Mampl.nc')
pzbM=xr.open_dataset('block_pvals_Mampl.nc')

pzto_st=pztO.stack(z={'latitude','longitude'})
pnew_to=pzto_st.t2m.copy(deep=True)
pnew_to.values=fdrcorrection(pnew_to.values,alpha=0.05, method='indep', is_sorted=False)[1]

pztm_st=pztM.stack(z={'latitude','longitude'})
pnew_tm=pztm_st.t2m.copy(deep=True)
pnew_tm.values=fdrcorrection(pnew_tm.values,alpha=0.05, method='indep', is_sorted=False)[1]

pzbo_st=pzbO.stack(z={'latitude','longitude'})
pzbo_drna=pzbo_st.dropna(dim='z')
pnew_bo=pzbo_st.block.copy(deep=True)
pnew_drnabo=pzbo_drna.block.copy(deep=True)
pzfdrbo=fdrcorrection(pzbo_drna.block.values,alpha=0.05, method='indep', is_sorted=False)[1]
pnew_drnabo.values=pzfdrbo
pnew_bo.loc[pnew_bo.z.isin(pnew_drnabo.z)]=pnew_drnabo

pzbm_st=pzbM.stack(z={'latitude','longitude'})
pzbm_drna=pzbm_st.dropna(dim='z')
pnew_bm=pzbm_st.block.copy(deep=True)
pnew_drnabm=pzbm_drna.block.copy(deep=True)
pzfdrbm=fdrcorrection(pzbm_drna.block.values,alpha=0.05, method='indep', is_sorted=False)[1]
pnew_drnabm.values=pzfdrbm
pnew_bm.loc[pnew_bm.z.isin(pnew_drnabm.z)]=pnew_drnabm

In [10]:
#Save to files
pnew_to.unstack().to_netcdf('t2m_pvals_Oampl_fdr.nc')
pnew_bo.unstack().to_netcdf('block_pvals_Oampl_fdr.nc')

pnew_tm.unstack().to_netcdf('t2m_pvals_Mampl_fdr.nc')
pnew_bm.unstack().to_netcdf('block_pvals_Mampl_fdr.nc')